In [1]:
x = {184737297734959104 : {'Name': ['adam'],
                        'avaliability': {'2023-06-15': 'yes',
                                        '2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-15': True,
                                 '2023-06-08': True,
                                '2023-06-01': True,
                                '2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-15': 1,
                                '2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}  
                            },

     462409072176791564 : {'Name': ['matty'],
                        'avaliability': {'2023-06-15': 'yes',
                                        '2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-15': True,
                                 '2023-06-08': True,
                                '2023-06-01': True,
                                '2023-05-25': True},
                        'goal': {'2023-06-15': 1,
                                '2023-06-08': 2,
                                '2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-15': 1,
                                 '2023-06-01': 0,
                                '2023-05-25': 1},
                        'motm-vote': {},
                        'motm': {'2023-06-08': 1}                            
                           },

     265573788232581120 : {'Name': ['tommy'],
                        'avaliability': {'2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'no'},
                        'paid': {'2023-06-08': True,
                                '2023-06-01': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            },   

     454252561063477250 : {'Name': ['noz', 'lawrence'],
                        'avaliability': {'2023-06-15': 'yes',
                                        '2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-15': True,
                                 '2023-06-08': True,
                                '2023-06-01': True,
                                        '2023-05-25': True},
                        'goal': {'2023-06-15': 1,
                                 '2023-06-08': 1,
                                 '2023-06-01': 0,
                                '2023-05-25': 1},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0}, 
                        'motm-vote': {},
                        'motm': {}      
                            },  

     462410416048308225 : {'Name': ['tom', 'Tom b'],
                        'avaliability': {'2023-06-01': 'no',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            },

     462654687225708555 : {'Name': ['tk', 'thomas'],
                        'avaliability': {'2023-06-15': 'yes',
                                        '2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-15': True,
                                 '2023-06-08': True,
                                '2023-06-01': True,
                                '2023-05-25': True},
                        'goal': {'2023-06-08': 2,
                                '2023-06-01': 1,
                                '2023-05-25': 1},
                        'assist': {'2023-06-15': 1,
                                 '2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}     
                            },

     265576326788808704 : {'Name': ['jack'],
                        'avaliability': {'2023-06-15': 'yes',
                                        '2023-06-08': 'yes',
                                        '2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-15': True,
                                 '2023-06-08': True,
                                '2023-06-01': True,
                                '2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            }

     }

In [1]:
# autoreload modules
%load_ext autoreload
%autoreload 2

In [2]:
import json

# read data from json
path = '//TRUENAS/Misc_storage/5aside_discord_bot/user_data/team_data.json'

# with open(path, 'r') as f:
#     data = json.load(f)

# save data to json
with open(path, 'w') as f:
    json.dump(x, f, indent=4)




In [5]:
from datetime import datetime as dt
meta_data = {
    'channel_id': {'test':1112672147412893696},
    'admin_id': [184737297734959104],
    'update_time': dt.time(18, 00).strftime('%H:%M'),
    'fixtures': {
        'last_attempt': dt(2023,6,17, 15, 0).strftime('%Y-%m-%d %H:%M:%S'),
        'last_success': dt(2023,6,17, 15, 0).strftime('%Y-%m-%d %H:%M:%S')
    },
    'chasers': {
        'available': dt(2023,6,17, 15, 0).strftime('%Y-%m-%d %H:%M:%S'),
        'paid': dt(2023,6,17, 15, 0).strftime('%Y-%m-%d %H:%M:%S'),
        'vote': dt(2023,6,17, 15, 0).strftime('%Y-%m-%d %H:%M:%S'),
    }
}

In [6]:
import json
# read data from json
path = '//TRUENAS/Misc_storage/5aside_discord_bot/meta_data.json'

# save data to json
with open(path, 'w') as f:
    json.dump(meta_data, f, indent=4)


In [19]:
from datetime import datetime as dt, timedelta
dt_now = dt(2023,6,26, 0, 0)
#dt_now = dt.now()
dt_now = dt_now.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
day = dt_now.weekday()

upcoming = dt_now + timedelta(days = (7 if day == 3 else ((3 - day) % 7)) )
current = dt_now + timedelta(days = - ((day - 3) % 7) )



0
2023-06-22 00:00:00
2023-06-29 00:00:00


In [12]:
(day-3)%7

2

In [8]:
(3-6)%7

4

In [1]:
a = ['test', 'test2']
b = ['test3', 'test4']
c = ['test5', 'test6']



TypeError: unsupported operand type(s) for +: 'int' and 'list'